<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import" data-toc-modified-id="Import-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import</a></span></li><li><span><a href="#Simulation-of-the-Ideal-$\cos(2\varphi)$-qubit" data-toc-modified-id="Simulation-of-the-Ideal-$\cos(2\varphi)$-qubit-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Simulation of the Ideal $\cos(2\varphi)$ qubit</a></span><ul class="toc-item"><li><span><a href="#Ideal-case" data-toc-modified-id="Ideal-case-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Ideal case</a></span><ul class="toc-item"><li><span><a href="#Charge-basis" data-toc-modified-id="Charge-basis-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Charge basis</a></span></li><li><span><a href="#Phase-basis" data-toc-modified-id="Phase-basis-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Phase basis</a></span></li></ul></li><li><span><a href="#Non-ideal-case" data-toc-modified-id="Non-ideal-case-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Non-ideal case</a></span><ul class="toc-item"><li><span><a href="#Charge-basis" data-toc-modified-id="Charge-basis-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Charge basis</a></span></li><li><span><a href="#Phase-basis" data-toc-modified-id="Phase-basis-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>Phase basis</a></span></li></ul></li><li><span><a href="#Circuit-that-implements-cos(2phi)" data-toc-modified-id="Circuit-that-implements-cos(2phi)-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Circuit that implements cos(2phi)</a></span></li></ul></li><li><span><a href="#Smith's-circuit" data-toc-modified-id="Smith's-circuit-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Smith's circuit</a></span><ul class="toc-item"><li><span><a href="#Effective-model" data-toc-modified-id="Effective-model-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Effective model</a></span></li><li><span><a href="#Exact-Hamiltonian" data-toc-modified-id="Exact-Hamiltonian-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Exact Hamiltonian</a></span></li></ul></li><li><span><a href="#Sai's-data" data-toc-modified-id="Sai's-data-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Sai's data</a></span></li></ul></div>

# Import

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from qutip import *
import sys
#here, configure the path to the directory for pre-defined functions.
sys.path.append('/Users/longnguyen/Documents/GitHub/Fluxonium_berkeley/')
import plotting_settings
from scipy.optimize import curve_fit
import matplotlib as mpl
mpl.rcParams['figure.dpi']= 300

# Simulation of the Ideal $\cos(2\varphi)$ qubit

In [ ]:
sqg = tensor(qeye(2), rx(np.pi/2))
gate = sqg*cphase(np.pi)*sqg
print (gate)
print (fidelity(gate, cnot()))

## Ideal case
### Charge basis

In [ ]:
#From qutip.org
def hamiltonian_charge(N, Ec, Ej, ng):
    """
    Return the charge qubit hamiltonian as a Qobj instance.
    """
    m = np.diag(4 * Ec * (np.arange(-N,N+1)-ng)**2) - 0.5 * Ej * (np.diag(np.ones(2*N-1), 2) + 
                                                               np.diag(np.ones(2*N-1), -2))
    return Qobj(m)

In [ ]:
#parameters
N = 20
Ec = 0.2
Ej = 16
lvl_num = 5
ng_array = np.linspace(-2,2,401)
energies = np.zeros((len(ng_array), 2*N+1))

#Spectrum
for idx_ng, ng in enumerate (ng_array):
    energies[idx_ng,:] = hamiltonian_charge(N, Ec, Ej, ng).eigenenergies()  

In [ ]:
plt.figure(figsize=[8,6])
for idx_lvl in range(0,N):
    plt.plot(ng_array, energies[:,idx_lvl]-np.min(energies[:,0]))
plt.ylim([-0.5,20])
plt.xlabel('$n_g$')
plt.ylabel('Frequency (GHz)')
plt.xlim([ng_array[0], ng_array[-1]])
# plt.savefig('CPB_spectrum1.png')

In [ ]:
#Wavefunctions in charge basis
ng = 0.0
energies = np.zeros(2*N+1)
wavefunction = np.zeros((2*N+1, 2*N+1))
energies, wavefunction = hamiltonian_charge(N, Ec, Ej, ng).eigenstates()   

In [ ]:
n = np.linspace(-N,N,2*N+1)
fig, ax = plt.subplots(1,2, figsize = [6,6], constrained_layout=True)
fig.tight_layout()
# plt.subplots_adjust(left = 0, right = 1.1)

state_to_plot = 0
ax[0].bar(n, abs(wavefunction[state_to_plot].full()[:,0]))
ax[0].set_xlabel('$n$')
ax[0].set_ylabel(r'$|\langle n|\psi \rangle | $')
ax[0].set_xlim([-10,10])

state_to_plot = 1
ax[1].bar(n, abs(wavefunction[state_to_plot].full()[:,0]), color = 'C1')
ax[1].set_xlabel('$n$')
# ax[1].set_ylabel(r'$|\langle n|$' + str(state_to_plot)+ r'$ \rangle |$')
ax[1].set_xlim([-10,10])


### Phase basis

In [ ]:
#Solve in phase basis
Nphi = 601
Ej = 16
Ec = 0.2
ng = 0
phi = np.linspace(-0.5,1.5,2*Nphi+1)*np.pi
phi_op = np.diag(phi)
dphi = phi[-1]-phi[-2]
dphi_coeff = -1.0j/(2*dphi)
n_op = dphi_coeff*(np.diag(np.ones(2*Nphi), 1) - np.diag(np.ones(2*Nphi), -1))
def hamiltonian_phase(Ec, Ej, ng):
    m = 4*Ec*(Qobj(n_op)-ng)**2 - Ej*Qobj(2*phi_op).cosm()
    return m
energies, wavefunction = hamiltonian_phase(Ec, Ej, ng).eigenstates()
    

In [ ]:
plt.figure(figsize =[6,6])
plt.plot(phi/np.pi, -Ej*np.cos(2*phi)-energies[0], color = 'k')
for state_to_plot in range(0,17,2):
    plt.plot(phi/np.pi, 2*np.real(wavefunction[state_to_plot].full()[:,0])*10+energies[state_to_plot]-energies[0], alpha = 0.5, linewidth = 0.5) 
plt.ylim([-8,14])
plt.xlabel('$\phi / \pi$')
plt.ylabel(r'$|\langle \varphi|\psi \rangle | $')
plt.xlim([phi[0]/np.pi, phi[-1]/np.pi])

## Non-ideal case 
### Charge basis


In [ ]:
#From qutip.org
def hamiltonian_charge(N, Ec, Ej1,Ej2, ng):
    """
    Return the charge qubit hamiltonian as a Qobj instance.
    """
    m = np.diag(4 * Ec * (np.arange(-N,N+1)-ng)**2) - 0.5 * Ej2 * (np.diag(np.ones(2*N-1), 2) + 
                                                               np.diag(np.ones(2*N-1), -2)) - 0.5 * Ej1 * (np.diag(np.ones(2*N), 1) + 
                                                               np.diag(np.ones(2*N), -1))
    return Qobj(m)

In [ ]:
#parameters
N = 20
Ec = 1
Ej2 = 10
Ej1 = 0.5
lvl_num = 5
ng_array = np.linspace(-2,2,401)
energies = np.zeros((len(ng_array), 2*N+1))

#Spectrum
for idx_ng, ng in enumerate (ng_array):
    energies[idx_ng,:] = hamiltonian_charge(N, Ec, Ej1, Ej2, ng).eigenenergies()  

In [ ]:
plt.figure(figsize=[8,6])
for idx_lvl in range(0,N):
    plt.plot(ng_array, energies[:,idx_lvl]-np.min(energies[:,0]))
plt.ylim([-0.5,20])
plt.xlabel('$n_g$')
plt.ylabel('Frequency (GHz)')
plt.xlim([ng_array[0], ng_array[-1]])
# plt.savefig('CPB_spectrum1.png')

In [ ]:
#Wavefunctions in charge basis
ng = 0.0
energies = np.zeros(2*N+1)
wavefunction = np.zeros((2*N+1, 2*N+1))
energies, wavefunction = hamiltonian_charge(N, Ec, Ej1, Ej2, ng).eigenstates()

In [ ]:
n = np.linspace(-N,N,2*N+1)
fig, ax = plt.subplots(1,2, figsize = [6,6], constrained_layout=True)
fig.tight_layout()
# plt.subplots_adjust(left = 0, right = 1.1)

state_to_plot = 0
ax[0].bar(n, abs(wavefunction[state_to_plot].full()[:,0]))
ax[0].set_xlabel('$n$')
ax[0].set_ylabel(r'$|\langle n|\psi \rangle | $')
ax[0].set_xlim([-10,10])

state_to_plot = 1
ax[1].bar(n, abs(wavefunction[state_to_plot].full()[:,0]), color = 'C1')
ax[1].set_xlabel('$n$')
# ax[1].set_ylabel(r'$|\langle n|$' + str(state_to_plot)+ r'$ \rangle |$')
ax[1].set_xlim([-10,10])

### Phase basis

In [ ]:
#Solve in phase basis
Nphi = 601
Ej2 = 10
Ej1 = 1
Ec = 1
ng = 0
phi = np.linspace(-0.5,1.5,2*Nphi+1)*np.pi
phi_op = np.diag(phi)
dphi = phi[-1]-phi[-2]
dphi_coeff = -1.0j/(2*dphi)
n_op = dphi_coeff*(np.diag(np.ones(2*Nphi), 1) - np.diag(np.ones(2*Nphi), -1))
def hamiltonian_phase(Ec, Ej1, Ej2, ng):
    m = 4*Ec*(Qobj(n_op)-ng)**2 - Ej1*Qobj(phi_op).cosm() - Ej2*Qobj(2*phi_op).cosm()
    return m
energies, wavefunction = hamiltonian_phase(Ec, Ej1, Ej2, ng).eigenstates()

In [ ]:
plt.figure(figsize =[6,6])
plt.plot(phi/np.pi, -Ej*np.cos(2*phi)-energies[0], color = 'k')
for state_to_plot in range(0,17,2):
    plt.plot(phi/np.pi, 2*np.real(wavefunction[state_to_plot].full()[:,0])*10+energies[state_to_plot]-energies[0], alpha = 0.5, linewidth = 0.5) 
plt.ylim([-8,14])
plt.xlabel('$\phi / \pi$')
plt.ylabel(r'$|\langle \varphi|\psi \rangle | $')
plt.xlim([phi[0]/np.pi, phi[-1]/np.pi])

## Circuit that implements cos(2phi)

In [ ]:
#SQUID
phi1 = np.linspace(-5,5,101)*np.pi
phi2 = np.linspace(-5,5,101)*np.pi
V = np.zeros((len(phi1), len(phi2)))
for idx1, p1 in enumerate(phi1):
    for idx2, p2 in enumerate(phi2):
        V[idx1, idx2] = -np.cos(p1/2) - np.cos(p2/2+np.pi)
plt.figure(figsize = [6,6])        
X,Y = np.meshgrid(phi1/np.pi,phi2/np.pi)
Z = V
plt.contour(X,Y,Z, 20, cmap = 'RdBu_r')

In [ ]:
def potential_smith(phi_sum, phi_dif, E_J, E_L, theta_ext):
    v = -2*E_J*np.cos(phi_sum)*np.cos(phi_dif) + E_L*(-phi_sum+theta_ext/2)**2 + E_L*(phi_dif+theta_ext/2)**2
    return v

E_L = 0.3
E_J = 6
theta_ext = np.pi
phi_sum_array = np.linspace(-3,3,601)*np.pi
phi_dif_array = np.linspace(-2,2,401)*np.pi
V1 = np.zeros((len(phi_sum_array), len(phi_dif_array)))
V2 = np.zeros((len(phi_sum_array), len(phi_dif_array)))
for idx1, p1 in enumerate(phi_sum_array):
    for idx2, p2 in enumerate(phi_dif_array):
        V1[idx1, idx2] = potential_smith(p1, p2, E_J, E_L, 0)
        V2[idx1, idx2] = potential_smith(p1, p2, E_J, E_L, np.pi)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = [9,3])

X,Y = np.meshgrid(phi_sum_array/np.pi, phi_dif_array/np.pi)
Z = V1.transpose()/E_J
# plt.contour(X,Y,Z, 50, cmap = 'RdBu_r')
ax1.pcolor(X,Y,Z,vmax = 6, vmin = -8, cmap = 'RdBu_r')
ax1.set_ylabel(r'$\varphi_\Delta/\pi$')
ax1.set_xlabel(r'$\varphi_\Sigma/\pi$')

Z = V2.transpose()/E_J
# plt.contour(X,Y,Z, 50, cmap = 'RdBu_r')
ax2.pcolor(X,Y,Z,vmax = 6, vmin = -3, cmap = 'RdBu_r')
# ax2.set_ylabel(r'$\varphi_\Delta/\pi$')
ax2.set_xlabel(r'$\varphi_\Sigma/\pi$')

# Smith's circuit
Emulate the spectrum of KITE qubit
## Effective model

In [ ]:
e = 1.602e-19    #Fundamental charge
h = 6.626e-34    #Placnk's constant
phi_o = h/(2*e) #Flux quantum

def bare_hamiltonian(N, E_l, E_c, E_j1, E_j2, phi_ext):
    a = destroy(N)
    phi = (a+a.dag())*(8.0*E_c/E_l)**(0.25)/np.sqrt(2.0)
    na = 1.0j*(a.dag()-a)*(E_l/(8*E_c))**(0.25)/np.sqrt(2.0)
    ope2 = 2.0j*(phi)
    ope1 = 1.0j*(phi)
    H = E_c*na**2.0 + 0.5*E_l*(phi-phi_ext)**2.0 - 0.5*E_j2*(ope2.expm() + (-ope2).expm()) - 0.5*E_j1*(ope1.expm() + (-ope1).expm())
    return H

In [ ]:
N = 100
E_l = 0.23
E_c = 2.5
E_j2 = 10.5
E_j1 = E_j2 * 0.00

phi_ext = np.linspace(0,1,51)
level_num = 10
energies = np.zeros((len(phi_ext),level_num))

# Compute eigensnergies
for idx, phi in enumerate(phi_ext):
    H = bare_hamiltonian(N, E_l, E_c, E_j1, E_j2, phi*2*np.pi)
    for idy in range(level_num):
        energies[idx,idy] = H.eigenenergies()[idy]
        
#Plot eigensnergies
for idx in range(4):
    plt.plot(phi_ext, energies[:,idx]-energies[:,0], linewidth = '2')
plt.ylim([0,5])
plt.xlim([0,1])
plt.ylabel('Transition energy (GHz)')
plt.xlabel(r'$\varphi_\mathrm{ext}/2\pi$')

## Exact Hamiltonian

In [ ]:
e = 1.602e-19    #Fundamental charge
h = 6.626e-34    #Placnk's constant
phi_o = h/(2*e) #Flux quantum

def exact_hamiltonian(N, E_l, E_cj, E_j, ep_c, ep_l, phi_ext, theta_ext):
    a1 = tensor(destroy(N),qeye(N), qeye(N))
    a2 = tensor(qeye(N), destroy(N), qeye(N))
    a3 = tensor(qeye(N), qeye(N), destroy(N))
    
    
    phi1 = (a1+a1.dag())*(8.0*E_cj/ep_l)**(0.25)/np.sqrt(2.0)
    na1 = 1.0j*(a1.dag()-a1)*(ep_l/(8 *E_cj))**(0.25)/np.sqrt(2.0)
    H1 = 4*E_cj*na1**2.0 + 0.5*ep_l*(phi1)**2.0 - E_j*phi1.cosm()
    
    phi2 = (a2+a2.dag())*(8.0*E_cj/ep_l)**(0.25)/np.sqrt(2.0)
    na2 = 1.0j*(a2.dag()-a2)*(ep_l/(8*E_cj))**(0.25)/np.sqrt(2.0)
    H2 = 4*E_cj*na2**2.0 + 0.5*ep_l*(phi2+theta_ext)**2.0 - E_j*phi2.cosm()
    
    phi3 = (a3+a3.dag())*(8.0*ep_c/E_l)**(0.25)/np.sqrt(2.0)
    na3 = 1.0j*(a3.dag()-a3)*(E_l/(8*ep_c))**(0.25)/np.sqrt(2.0)
    H3 = 4*ep_c*na3**2.0 + 0.5*E_l*(phi3-phi_ext)**2.0
    
    H4 = ep_l*phi1*(phi2+phi3+theta_ext)
    
    H = H1 + H2 + H3 + H4
    return H

In [ ]:
phi_ext = np.linspace(-0.2, 1.2,41)*np.pi*2
energies = np.zeros((len(phi_ext), 5))

N=20
E_j = 5.9
E_c = 6.6
E_l = 0.23
ep_l = 0.36
ep_c = 2.5
theta_ext = np.pi

for idx, phi in enumerate(phi_ext):
    H = exact_hamiltonian(N, E_l, E_c, E_j, ep_c, ep_l, phi, theta_ext)
    eigen_energies = H.eigenenergies()
    for idy in range(5):
        energies[idx,idy] = eigen_energies[idy+1] - eigen_energies[0]
    print (idx)

In [ ]:
for idy in range(5):
    plt.plot(phi_ext/(2*np.pi), energies[:,idy])

# Sai's data

In [ ]:
data = np.array([0.000000000000000000e+00, 2.643380480358479279e+00, 4.056178964939483045e+00, 5.526766312567801265e+00, 5.999450334455434408e+00, 7.912061466620323280e+00, 8.254797463653288148e+00,
3.333333333333333287e-02, 2.390635755622190217e+00, 3.805016235639274491e+00, 5.251743056623653594e+00, 5.779360649467251143e+00, 7.640856409816592532e+00, 7.891795809452099242e+00,
6.666666666666666574e-02, 2.106158667385723327e+00, 3.525056224966625518e+00, 4.914602423193579206e+00, 5.578769287475540573e+00, 7.284135237388012385e+00, 7.512879372715292270e+00,
1.000000000000000056e-01, 1.789989875784279150e+00, 3.215664128382334841e+00, 4.536761706309195397e+00, 5.376611280505529500e+00, 6.834451826268071351e+00, 7.149579795333012022e+00,
1.333333333333333315e-01, 1.442188867588245271e+00, 2.875833687124674132e+00, 4.130933884360272756e+00, 5.160627994647416728e+00, 6.324584035455517395e+00, 6.778518162429662652e+00,
1.666666666666666574e-01, 1.062823405687021827e+00, 2.504291981248799370e+00, 3.703764361450008646e+00, 4.924361244545353067e+00, 5.775450545022994042e+00, 6.385072587269893241e+00,
2.000000000000000111e-01, 6.519577426966144396e-01, 2.099686660215847933e+00, 3.259114070602735946e+00, 4.662466827068945996e+00, 5.196794168366817956e+00, 5.965460165475247045e+00,
2.333333333333333370e-01, 2.096416735267694620e-01, 1.660855418186999222e+00, 2.799337399851031360e+00, 4.358644676476648883e+00, 4.606349673114428889e+00, 5.518836524587605652e+00,
2.666666666666666630e-01, -2.640977960246627676e-01, 1.187118059747610754e+00, 2.325602119173547866e+00, 3.884900621084939942e+00, 4.132591426162660397e+00, 5.045048324981451060e+00,
2.999999999999999889e-01, -7.692609126338498804e-01, 6.784739612648631457e-01, 1.837908185506730518e+00, 3.241223927684192763e+00, 3.775531518845769874e+00, 4.544101262296003085e+00,
3.333333333333333148e-01, -1.305875136925316982e+00, 1.356022230832096553e-01, 1.335087538979127020e+00, 2.555619276769847570e+00, 3.406687939566919621e+00, 4.016159596146699151e+00,
3.666666666666666408e-01, -1.873990608749754205e+00, -4.403355559630614535e-01, 8.147848951668473783e-01, 1.844390455280138452e+00, 3.008325630215751278e+00, 3.462077116787031006e+00,
4.000000000000000222e-01, -2.473671760848192402e+00, -1.047987257438926845e+00, 2.731373044892912993e-01, 1.112884148931533757e+00, 2.570703886410749650e+00, 2.885643536749235682e+00,
4.333333333333333481e-01, -3.104986341689570661e+00, -1.686079760226590674e+00, -2.965048261466805002e-01, 3.675625843226328349e-01, 2.072905224150784154e+00, 2.301485192929330292e+00,
4.666666666666666741e-01, -3.767993629954179635e+00, -2.353606273305689722e+00, -9.068623476769964720e-01, -3.793082705947440347e-01, 1.482163374727833993e+00, 1.732969980954969458e+00,
5.000000000000000000e-01, -4.462733916267845657e+00, -3.049931238627821006e+00, -1.579357664167319930e+00, -1.106659137384933489e+00, 8.059462829131089912e-01, 1.148659037106348979e+00,
5.333333333333333259e-01, -5.189221241937573836e+00, -3.774837984751309161e+00, -2.328140333610034141e+00, -1.800468791017214132e+00, 6.103452122360489285e-02, 3.120208916280275968e-01,
5.666666666666666519e-01, -5.947440842449983833e+00, -4.528541846369704693e+00, -3.139031110979102923e+00, -2.474789594088730116e+00, -7.694096271366538398e-01, -5.406022862223589520e-01,
5.999999999999999778e-01, -6.737351820928807022e+00, -5.311677175092842873e+00, -3.990614789112341132e+00, -3.150684924297896927e+00, -1.692830359188380429e+00, -1.377614458303108025e+00,
6.333333333333333037e-01, -7.558894506489182596e+00, -6.125249835084614070e+00, -4.870181857872325715e+00, -3.840410387789141122e+00, -2.676440591219781506e+00, -2.222404941485957686e+00,
6.666666666666666297e-01, -8.412001033852316567e+00, -6.970532474693905023e+00, -5.771088766892000343e+00, -4.550422657983824948e+00, -3.699319589309697953e+00, -3.089597137631645563e+00,
6.999999999999999556e-01, -9.296607145388581017e+00, -7.848877367225824742e+00, -6.689475514291469693e+00, -5.286065271101447749e+00, -4.751723378947454002e+00, -3.982972580206292967e+00,
7.333333333333332815e-01, -1.021266313866197883e+01, -8.761446989780974803e+00, -7.622988165171293851e+00, -6.063638034128607579e+00, -5.815916126948622811e+00, -4.903372458411199730e+00,
7.666666666666666075e-01, -1.116014218282592019e+01, -9.708921955628669664e+00, -8.570459117328235621e+00, -7.011125937301629030e+00, -6.763431446928827029e+00, -5.850945644771792686e+00,
8.000000000000000444e-01, -1.213904475441706055e+01, -1.069130346951139288e+01, -9.531888453943590633e+00, -8.128549716024210170e+00, -7.594245979383518197e+00, -6.825681133971436942e+00,
8.333333333333333703e-01, -1.314939856293683995e+01, -1.170791303364464575e+01, -1.050844433721222515e+01, -9.287904078848397305e+00, -8.436840878007663136e+00, -7.827408954466524627e+00,
8.666666666666666963e-01, -1.419125397719570358e+01, -1.275758953675038043e+01, -1.150248243365081358e+01, -1.047288202917111732e+01, -9.308952536765888297e+00, -8.855269723453659481e+00,
9.000000000000000222e-01, -1.526467560280460312e+01, -1.383898115393543726e+01, -1.251786664070234423e+01, -1.167813523455038194e+01, -1.022032085356290310e+01, -9.905468760868336275e+00,
9.333333333333333481e-01, -1.636973127719004495e+01, -1.495081484823716345e+01, -1.356124860102946528e+01, -1.289719921453531626e+01, -1.119186425031725562e+01, -1.096337407590319835e+01,
9.666666666666666741e-01, -1.750648026924223544e+01, -1.609208372725861835e+01, -1.464535303293191504e+01, -1.411780419514623475e+01, -1.225634743681026428e+01, -1.200561762731176785e+01,
1.000000000000000000e+00, -1.867496276199383942e+01, -1.726215198371976456e+01, -1.579160501539533001e+01, -1.531887847170053973e+01, -1.340628406258280236e+01, -1.306361615387210762e+01])

In [ ]:
flux = data[0::7]
e0 = data[1::7]
e1 = data[2::7]
e2 = data[3::7]
e3 = data[4::7]
e4 = data[5::7]
e5 = data[6::7]

# plt.plo
plt.plot(flux, e1-e0)
plt.plot(flux, e2-e0)
plt.plot(flux, e3-e0)
plt.plot(flux, e4-e0)
plt.plot(flux, e5-e0)
plt.xlabel('Flux')
plt.ylabel('Energies')

In [ ]:
N = 50
def fit_energy(phi_ext, E_l, E_c, E_j1, E_j2):
    energy1 = np.zeros_like(flux)
    energy2 = np.zeros_like(flux)
    energy3 = np.zeros_like(flux)
#     energy4 = np.zeros_like(flux)
    
    a = destroy(N)
    phi = (a+a.dag())*(E_c/(2*E_l))**(0.25)/np.sqrt(2.0)
    na = 1.0j*(a.dag()-a)*(2*E_l/(E_c))**(0.25)/np.sqrt(2.0)
    ope2 = 2.0*(phi)
    ope1 = 1.0*(phi)
    for idx in range(len(flux)):
        H = E_c*na**2.0 + 0.5*E_l*(phi-flux[idx]*2*np.pi)**2.0 + E_j2*ope2.cosm() - E_j1*ope1.cosm()
        energies = H.eigenenergies()
        energy1[idx] = energies[1] - energies[0]
        energy2[idx] = energies[2] - energies[0]
        energy3[idx] = energies[3] - energies[0]
#         energy4[idx] = energies[4] - energies[0]
    return np.concatenate([energy1, energy2,energy3], axis=0)
guess = [0.16, 6.6, 0, 5.9]
opt, cov = curve_fit(fit_energy, xdata = np.concatenate([flux,flux ,flux], axis=0),
                     ydata=np.concatenate([e1-e0,e2-e0 ,e3-e0], axis=0), p0 = guess)
print (opt)

In [ ]:
def bare_hamiltonian(N, E_l, E_c, E_j1, E_j2, phi_ext):
    a = destroy(N)
    phi = (a+a.dag())*(E_c/(2*E_l))**(0.25)/np.sqrt(2.0)
    na = 1.0j*(a.dag()-a)*(2*E_l/(E_c))**(0.25)/np.sqrt(2.0)
    ope2 = 2.0*(phi)
    ope1 = 1.0*(phi)
    H = E_c*na**2.0 + 0.5*E_l*(phi+phi_ext)**2.0 + E_j2*ope2.cosm() - E_j1*ope1.cosm()
    return H
#Data
plt.errorbar(flux, e1-e0, linestyle='none', marker='d', mfc='none', ecolor = 'C0', mec='C0', ms=3,mew=1)
plt.errorbar(flux, e2-e0,linestyle='none', marker='d', mfc='none', ecolor = 'C1', mec='C1', ms=3,mew=1)
plt.errorbar(flux, e3-e0,linestyle='none', marker='d', mfc='none', ecolor = 'C2', mec='C2', ms=3,mew=1)
plt.errorbar(flux, e4-e0,linestyle='none', marker='d', mfc='none', ecolor = 'C3', mec='C3', ms=3,mew=1)
plt.errorbar(flux, e5-e0,linestyle='none', marker='d', mfc='none', ecolor = 'C4', mec='C4', ms=3,mew=1)

#Fit
E_l, E_c, E_j1, E_j2 = opt
# E_l, E_c, E_j1, E_j2 = [0.16, 6.6, 0, 5.9]
phi_ext = np.linspace(0,1,51)
level_num = 10
energies = np.zeros((len(phi_ext),level_num))

# Compute eigensnergies
for idx, phi in enumerate(phi_ext):
    H = bare_hamiltonian(N, E_l, E_c, E_j1, E_j2, phi*2*np.pi)
    for idy in range(level_num):
        energies[idx,idy] = H.eigenenergies()[idy]
        
#Plot eigensnergies
for idx in range(5):
    plt.plot(phi_ext, energies[:,idx]-energies[:,0], linewidth = '2')
plt.ylim([0,5])
plt.xlim([0,1])
plt.ylabel('Transition energy (GHz)')
plt.xlabel(r'$\varphi_\mathrm{ext}/2\pi$')

In [ ]:
0.995**4